# Structure Collection

In [1]:
from typing import ClassVar
from byteclasses import ByteOrder
from byteclasses.types.collections import FixedSizeCollection, structure, member
from byteclasses.types.primitives.characters import UChar
from byteclasses.types.primitives.generics import Byte
from byteclasses.types.primitives.integers import Int8, UInt8, UInt16, UInt32
from byteclasses.types.primitives.floats import Float32

Basic Structure

In [2]:
@structure
class BasicStructure:
    """A basic structure byteclass."""

    var1: UInt8 = UInt8()
    var2: UInt16 = UInt16()


bs = BasicStructure()
print(bs, len(bs), bs.data)

BasicStructure(var1=UInt8(0), var2=UInt16(0)) 4 bytearray(b'\x00\x00\x00\x00')


Packed Basic Structure

In [3]:
@structure(packed=True)
class PackedStructure:
    """A Packed structure byteclass."""

    var1: UInt8 = UInt8()
    var2: UInt16 = UInt16()


ps = PackedStructure()
print(ps, len(ps), ps.data)

PackedStructure(var1=UInt8(0), var2=UInt16(0)) 3 bytearray(b'\x00\x00\x00')


Simple Structure

In [4]:
@structure(byte_order=b"!")
class SimpleStruct():
    a:UInt8
    b:UInt16
    c:UChar
    d:Float32
    e: UInt8
ss1: FixedSizeCollection = SimpleStruct()
print(len(ss1))
print(ss1)
print(ss1.data)

13
SimpleStruct(a=UInt8(0), b=UInt16(0), c=UChar('\x00'), d=Float32(0.0), e=UInt8(0))
bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


Direct data access

In [5]:
ss1.data = b"\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00"
print(ss1.data)
print(ss1.a)
print(ss1.b)

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
0
0


In [6]:
ss2 = SimpleStruct(a=UInt8(1), b=UInt16(2), c=UInt8(3), d=Float32(4))
print(ss2)
print(ss2.data)

SimpleStruct(a=UInt8(1), b=UInt16(512), c=UInt8(3), d=Float32(4.600743118071239e-41), e=UInt8(0))
bytearray(b'\x01\x00\x02\x00\x03\x00\x00\x00\x00\x00\x80@\x00')


Member Assignment

In [7]:
print(ss1.a.endianness)
ss1.a = 1
print(ss1.a.data)
print(ss1.data)
ss1.b = 2
print(ss1.b.data)
print(ss1.data)
ss1.c = "c"
print(ss1.c.data)
print(ss1.data)
ss1.d = 4
print(ss1.d.data)
print(ss1.data)

NET
b'\x01'
bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
b'\x00\x02'
bytearray(b'\x01\x00\x00\x02\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
b'c'
bytearray(b'\x01\x00\x00\x02c\x00\x00\x00\xcd\xcc\x8c?\x00')
b'@\x80\x00\x00'
bytearray(b'\x01\x00\x00\x02c\x00\x00\x00@\x80\x00\x00\x00')


Index Assignment

In [8]:
ss1[1] = 6
ss1[2] = 7
ss1[3] = 8
print(ss1.data)
print(ss1)

bytearray(b'\x01\x06\x07\x08c\x00\x00\x00@\x80\x00\x00\x00')
SimpleStruct(a=UInt8(1), b=UInt16(1800), c=UChar('c'), d=Float32(4.0), e=UInt8(0))


Nested Collections

In [9]:
@structure
class InnerStruct:
    var0: ClassVar[int] = 23
    var2: Byte = member(default_factory=Byte)

In [10]:
@structure(byte_order=ByteOrder.NATIVE, packed=True)
class OuterStruct:
    var3: ClassVar[int] = 23
    var4: Int8 = member(default_factory=Int8)
    var5: UInt32 = member(default=UInt32(65535))
    var6: Float32 = Float32(1.1)
    var7: InnerStruct = member(default_factory=InnerStruct)

    def test(self):
        print("test")

In [11]:
test = OuterStruct()
print(test)
print(test.var7)
print(bytes(test))

OuterStruct(var4=Int8(0), var5=UInt32(65535), var6=Float32(1.100000023841858), var7=InnerStruct(var2=Byte(data=b'\x00', byte_order=b'@')))
InnerStruct(var2=Byte(data=b'\x00', byte_order=b'@'))
b'\x00\xff\xff\x00\x00\xcd\xcc\x8c?\x00'


Member Assignment

In [12]:
test.var4 = 1
print(test)
print(test.data)

OuterStruct(var4=Int8(1), var5=UInt32(65535), var6=Float32(1.100000023841858), var7=InnerStruct(var2=Byte(data=b'\x00', byte_order=b'@')))
bytearray(b'\x01\xff\xff\x00\x00\xcd\xcc\x8c?\x00')


Data Attribute Assignment

In [13]:
test.data = b"\xf9\xf8\xf7\xf6\xf5\xf4\xf3\xf2\xf1\xf0"
print(test)
print(test.data)

OuterStruct(var4=Int8(-7), var5=UInt32(4126603256), var6=Float32(-2.4060893954673178e+30), var7=InnerStruct(var2=Byte(data=b'\xf0', byte_order=b'@')))
bytearray(b'\xf9\xf8\xf7\xf6\xf5\xf4\xf3\xf2\xf1\xf0')


Slice Assignment

In [14]:
test[2:5] = b"\x08"
print(test)
print(test.data)

OuterStruct(var4=Int8(-7), var5=UInt32(134744312), var6=Float32(-2.4060893954673178e+30), var7=InnerStruct(var2=Byte(data=b'\xf0', byte_order=b'@')))
bytearray(b'\xf9\xf8\x08\x08\x08\xf4\xf3\xf2\xf1\xf0')
